# **Inference/Testing**

### **Import Library**

In [1]:
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

### **Load Dataset**

In [2]:
data = pd.read_csv('Data\Stress.csv')

In [3]:
data

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1
...,...,...
2833,"* Her, a week ago: Precious, how are you? (I i...",0
2834,I don't have the ability to cope with it anymo...,1
2835,In case this is the first time you're reading ...,0
2836,Do you find this normal? They have a good rela...,0


In [4]:
data['text']

0       He said he had not felt that way before, sugge...
1       Hey there r/assistance, Not sure if this is th...
2       My mom then hit me with the newspaper and it s...
3       until i met my new boyfriend, he is amazing, h...
4       October is Domestic Violence Awareness Month a...
                              ...                        
2833    * Her, a week ago: Precious, how are you? (I i...
2834    I don't have the ability to cope with it anymo...
2835    In case this is the first time you're reading ...
2836    Do you find this normal? They have a good rela...
2837    I was talking to my mom this morning and she s...
Name: text, Length: 2838, dtype: object

### **Mengambil data yang akan ditest**

Mengambil data dari text ke 2 dan menginisialisasikan ke dalam text_test untuk proses test

In [5]:
text_test = data['text'][1]

text_test

"Hey there r/assistance, Not sure if this is the right place to post this.. but here goes =) I'm currently a student intern at Sandia National Labs and working on a survey to help improve our marketing outreach efforts at the many schools we recruit at around the country. We're looking for current undergrad/grad STEM students so if you're a STEM student or know STEM students, I would greatly appreciate if you can help take or pass along this short survey. As a thank you, everyone who helps take the survey will be entered in to a drawing for chance to win one of three $50 Amazon gcs."

### **Prediksi**

Menyiapkan data input dalam format yang sesuai untuk prediksi menggunakan model yang telah dilatih, kemudian mengirimkan permintaan prediksi ke endpoint model TensorFlow Serving. Fungsi `prepare_json(text)` mempersiapkan data teks yang akan diprediksi, mengonversinya ke format `tf.train.Example` dan membungkus dalam format JSON sesuai dengan kebutuhan API TensorFlow Serving. Data teks yang diberikan diubah menjadi format bytes, diserialisasi, kemudian dikodekan dalam Base64 dan dimasukkan ke dalam struktur JSON dengan kunci `signature_name` dan `instances`.

Setelah JSON siap, kode ini mengirimkan permintaan HTTP POST ke endpoint `http://localhost:8080/v1/models/stress-prediction-model:predict` dengan data JSON menggunakan `requests.post(endpoint, data=json_data)`. Respons dari server kemudian diproses untuk mengambil nilai prediksi dan menentukan hasilnya. Jika nilai prediksi lebih besar dari 0.6, hasilnya adalah "Stress", jika lebih kecil atau sama dengan 0.6, hasilnya adalah "Tidak Stress". Jika tidak ada prediksi ditemukan dalam respons, maka hasil akan menampilkan pesan kesalahan.

In [6]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])),
    }
    
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

json_data = prepare_json(text_test)
    
endpoint = "http://localhost:8080/v1/models/stress-prediction-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Stress" if prediction_value > 0.6 else "Tidak Stress"
else:
    result = "Error: No predictions found in response."

print(result)

Tidak Stress
